## Preprocessing

In [15]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.3 MB/s eta 0:00:00


In [42]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from google.colab import files

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [17]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN", "NAME"], inplace=True)

In [18]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [19]:
# Look at APPLICATION_TYPE value counts for binning
app_value_counts = application_df["APPLICATION_TYPE"].value_counts()
app_value_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [20]:
# Choose a cutoff value of 100 and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = app_value_counts.loc[lambda x : x<100].index.to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [21]:
# Look at CLASSIFICATION value counts for binning
class_value_counts = application_df["CLASSIFICATION"].value_counts()
class_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [22]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_value_counts.loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [23]:
# Choose a cutoff value of 100 and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = class_value_counts.loc[lambda x : x<100].index.to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [24]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummy_df = pd.get_dummies(application_df)

In [25]:
# Split our preprocessed data into our features and target arrays
y = application_dummy_df["IS_SUCCESSFUL"]
X = application_dummy_df.drop(columns = "IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [26]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# find number of columns to find the input dimension
len(X.columns)

50

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=50))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                510       
                                                                 
 dense_5 (Dense)             (None, 10)                110       
                                                                 
 dense_6 (Dense)             (None, 10)                110       
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 741
Trainable params: 741
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6029 - accuracy: 0.6987
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5606 - accuracy: 0.7271
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5548 - accuracy: 0.7296
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7304
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7303
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5488 - accuracy: 0.7320
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7313
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7331
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accuracy: 0.7321
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accura

In [28]:
# Evaluate the model using the test data
model_loss_1, model_accuracy_1 = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_1}, Accuracy: {model_accuracy_1}")

268/268 - 1s - loss: 0.5534 - accuracy: 0.7278 - 729ms/epoch - 3ms/step
Loss: 0.5533956289291382, Accuracy: 0.7278134226799011


In [20]:
# Define the second attempt at a model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(tf.keras.layers.Dense(units=10, activation="tanh", input_dim=50))

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=10, activation="tanh"))

# Third hidden layer
nn_2.add(tf.keras.layers.Dense(units=10, activation="tanh"))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 10)                510       
                                                                 
 dense_9 (Dense)             (None, 10)                110       
                                                                 
 dense_10 (Dense)            (None, 10)                110       
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
Total params: 741
Trainable params: 741
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the second model
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Train the second model
fit_model_2 = nn_2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 2ms/step - loss: 0.5944 - accuracy: 0.6929
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5594 - accuracy: 0.7284
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5545 - accuracy: 0.7292
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5520 - accuracy: 0.7303
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5505 - accuracy: 0.7322
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7310
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7296
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7322
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5469 - accuracy: 0.7325
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5461 - accura

In [29]:
# Evaluate the model using the test data
model_loss_2, model_accuracy_2 = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_2}, Accuracy: {model_accuracy_2}")

268/268 - 1s - loss: 0.5544 - accuracy: 0.7249 - 765ms/epoch - 3ms/step
Loss: 0.5543796420097351, Accuracy: 0.7248979806900024


In [24]:
# Define the third attempt at a model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_3 = tf.keras.models.Sequential()

# First hidden layer
nn_3.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=50))

# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Third hidden layer
nn_3.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Fourth hidden layer
nn_3.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Fifth hidden layer
nn_3.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 10)                510       
                                                                 
 dense_13 (Dense)            (None, 10)                110       
                                                                 
 dense_14 (Dense)            (None, 10)                110       
                                                                 
 dense_15 (Dense)            (None, 10)                110       
                                                                 
 dense_16 (Dense)            (None, 10)                110       
                                                                 
 dense_17 (Dense)            (None, 1)                 11        
                                                                 
Total params: 961
Trainable params: 961
Non-trainable 

In [25]:
# Compile the second model
nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the second model
fit_model_3 = nn_3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 5s 3ms/step - loss: 0.5944 - accuracy: 0.7001
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5583 - accuracy: 0.7266
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7292
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5507 - accuracy: 0.7311
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7313
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7319
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5477 - accuracy: 0.7306
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5470 - accuracy: 0.7322
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7316
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - accura

In [30]:
# Evaluate the model using the test data
model_loss_3, model_accuracy_3 = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_3}, Accuracy: {model_accuracy_3}")

268/268 - 0s - loss: 0.5554 - accuracy: 0.7237 - 370ms/epoch - 1ms/step
Loss: 0.5553731322288513, Accuracy: 0.7237317562103271


In [33]:
print(f"First attempt has an accuracy of {model_accuracy_1}")
print(f"Second attempt has an accuracy of {model_accuracy_2}")
print(f"Third attempt has an accuracy of {model_accuracy_3}")

First attempt has an accuracy of 0.7278134226799011
Second attempt has an accuracy of 0.7248979806900024
Third attempt has an accuracy of 0.7237317562103271


In [27]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=5), activation=activation, input_dim=50))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 7)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [28]:
# create a tuner object for finding the best hyperparameters

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [29]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 01m 24s]
val_accuracy: 0.532711386680603

Best val_accuracy So Far: 0.7325947284698486
Total elapsed time: 00h 25m 15s


In [30]:
# Find the best three models
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5530 - accuracy: 0.7326 - 600ms/epoch - 2ms/step
Loss: 0.5529996156692505, Accuracy: 0.7325947284698486
268/268 - 2s - loss: 0.5541 - accuracy: 0.7320 - 2s/epoch - 9ms/step
Loss: 0.554059624671936, Accuracy: 0.7320116758346558
268/268 - 1s - loss: 0.5515 - accuracy: 0.7317 - 1s/epoch - 4ms/step
Loss: 0.5515387058258057, Accuracy: 0.7316617965698242


In [33]:
# Find top 3 model hyperparameters and print the values
top_hypers = tuner.get_best_hyperparameters(3)
for param in top_hypers:
    print(param.values)

{'activation': 'tanh', 'first_units': 21, 'num_layers': 6, 'units_0': 26, 'units_1': 1, 'units_2': 11, 'units_3': 16, 'units_4': 1, 'units_5': 26, 'units_6': 6, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0042'}
{'activation': 'tanh', 'first_units': 16, 'num_layers': 5, 'units_0': 11, 'units_1': 26, 'units_2': 21, 'units_3': 6, 'units_4': 6, 'units_5': 1, 'units_6': 6, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0012'}
{'activation': 'relu', 'first_units': 21, 'num_layers': 1, 'units_0': 16, 'units_1': 11, 'units_2': 21, 'units_3': 1, 'units_4': 26, 'units_5': 1, 'units_6': 16, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0019'}


In [37]:
top_model = tuner.get_best_models(1)

In [40]:
top_model[0].save("AlphabetSoupCharity_Optimization.h5")

In [43]:
files.download("AlphabetSoupCharity_Optimization.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>